In [2]:
import pandas as pd
import numpy as np
import pyodbc
import streamlit as st

# 1. Connect to the server

In [2]:
server = "."
database = "Northwind"

#cnxn is represent a database connection object created by 'pyodbc.connect()' function
#So it basically connects to the database
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')


# 2. Read the data from the database 

In [3]:
#Create a dataframe and read it directly from a query, and with the connection object = cnxn
#Assigning pandas dataframe to a variable df
df = pd.read_sql_query('SELECT * FROM Northwind.dbo.Customers', cnxn)
df

C:\Users\User\AppData\Local\Temp\ipykernel_8336\2019294039.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('SELECT * FROM Northwind.dbo.Customers', cnxn)


,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...
86,WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
87,VICTE,Victuailles en stock,Mary Saveley,Sales Agent,"2, rue du Commerce",Lyon,None,69004,France,78.32.54.86,78.32.54.87
88,WILMK,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858
89,VINET,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11


# 3. Inspecting and identify outliers
In this section, i will be inspecting any abnormalities of the data and note it under *'Handle outliers'* heading. <br>



In [4]:
#Inspecting data from the first 15 rows
df.head(5)

#CONCLUSION: name must be separated, Address must be break down, Postal code must be cleaned up

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [5]:
#Check over all info of every columns in the dataframe
df.info()

#CONCLUSION: We can see that the data type of the columns are almost all object types, must convert them to the correct data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    91 non-null     object
 1   CompanyName   91 non-null     object
 2   ContactName   91 non-null     object
 3   ContactTitle  91 non-null     object
 4   Address       91 non-null     object
 5   City          91 non-null     object
 6   Region        31 non-null     object
 7   PostalCode    90 non-null     object
 8   Country       91 non-null     object
 9   Phone         91 non-null     object
 10  Fax           69 non-null     object
dtypes: object(11)
memory usage: 7.9+ KB


In [6]:
#Check if there is null in any column
df.isnull().sum()
#CONCLUSION: There are null values in 'Region' and 'Fax' columns

CustomerID       0
CompanyName      0
ContactName      0
ContactTitle     0
Address          0
City             0
Region          60
PostalCode       1
Country          0
Phone            0
Fax             22
dtype: int64

In [7]:
#Looking for a duplicate row
df.duplicated().sum()
#CONCLUSION: There is no duplicate row

0

# 4. Handle outliers
This are the list of abnormalities in the table of dbo.Customers:
1. We can make a 'CustomerID' column to be the indexed column. **DONE**
2. We can see that Customer's name are in the same cell, which mean it will harder for analyzing, must separate firstname and lastname. **DONE**   
3. Address column must be separated by street and the address itselves. 
4. Changing most of the columns data type to its suitable data type.
5. Postal code must be cleaned up

1. We can make a 'CustomerID' column to be the indexed column. **DONE**


In [8]:
#Set the column 'CustomerID' as the index
df.set_index('CustomerID', inplace=True)
#inplace is the parameter that will change the dataframe itself, not returning a new one. And change is permanent.

df

,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
CustomerID,,,,,,,,,,
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...
WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
VICTE,Victuailles en stock,Mary Saveley,Sales Agent,"2, rue du Commerce",Lyon,None,69004,France,78.32.54.86,78.32.54.87
WILMK,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


2. We can see that Customer's name are in the same cell, which mean it will harder for analyzing, must separate firstname and lastname. **DONE**   


In [9]:
#Split the 'ContactName' column into 'FirstName' and 'LastName'
#The expand parameter is for returning a DataFrame with splitted strings
#the n parameter is for the number of splits to do
df[['FirstName', 'LastName']] = df['ContactName'].str.split(' ', n=1, expand=True)
df

,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,FirstName,LastName
CustomerID,,,,,,,,,,,,
ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,Maria,Anders
ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745,Ana,Trujillo
ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None,Antonio,Moreno
AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750,Thomas,Hardy
BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67,Christina,Berglund
...,...,...,...,...,...,...,...,...,...,...,...,...
WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115,Karl,Jablonski
VICTE,Victuailles en stock,Mary Saveley,Sales Agent,"2, rue du Commerce",Lyon,None,69004,France,78.32.54.86,78.32.54.87,Mary,Saveley
WILMK,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858,Matti,Karttunen


In [10]:
#Get the index of the 'ContactName' column
contact_name_index = df.columns.get_loc('ContactName')

#[df.columns[:contact_name_index + 1] -> slice the columns from the beginning to the 'ContactName' column
#['FirstName', 'LastName'] -> add the new columns 'FirstName' and 'LastName'
#[df.columns[contact_name_index + 1:-2] -> slice the columns from the 'ContactName' column to the last two columns of DataFrame
#So basically it skips the 'ContactName' column and add the new columns 'FirstName' and 'LastName' in the middle of the DataFrame
new_column_order = list(df.columns[:contact_name_index + 1]) + ['FirstName', 'LastName'] + list(df.columns[contact_name_index + 1:-2])

#remove the 'ContactName' column from the list
new_column_order.remove('ContactName')

#Apply the new column order to the dataframe
df = df[new_column_order]

df

,CompanyName,FirstName,LastName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
CustomerID,,,,,,,,,,,
ALFKI,Alfreds Futterkiste,Maria,Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana,Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio,Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
AROUT,Around the Horn,Thomas,Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina,Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...
WHITC,White Clover Markets,Karl,Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
VICTE,Victuailles en stock,Mary,Saveley,Sales Agent,"2, rue du Commerce",Lyon,None,69004,France,78.32.54.86,78.32.54.87
WILMK,Wilman Kala,Matti,Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


3. Address column must be separated by street and the address itselves. 


In [11]:
#Separate the address from the street that contain int in the 'Address' column
street_num = df['Address'].str.extract('(\d+)')


#Assign a data to a new column called StreetNumber
df['StreetNumber'] = street_num


#Remove the street number from the 'Address' column by replacing it with an empty string with regex
df['Address'] = df['Address'].astype(str).str.replace('\d+', '', regex=True)

#Move the street number column to the column behind Address column
address_index = df.columns.get_loc('Address')

#Marking the list from beginning of column untill 'Address' column
#Force/add in a column StreetNumber in between the rest of the column
#Marking a list of columns again untill the last column of the DataFrame 
new_column_order = list(df.columns[:address_index + 1]) + ['StreetNumber'] + list(df.columns[address_index + 1:])



if new_column_order.count('StreetNumber') > 1:
    new_column_order.reverse()
    new_column_order.remove('StreetNumber')
    new_column_order.reverse()

df = df[new_column_order]
df

,CompanyName,FirstName,LastName,ContactTitle,Address,StreetNumber,City,Region,PostalCode,Country,Phone,Fax
CustomerID,,,,,,,,,,,,
ALFKI,Alfreds Futterkiste,Maria,Anders,Sales Representative,Obere Str.,57,Berlin,None,12209,Germany,030-0074321,030-0076545
ANATR,Ana Trujillo Emparedados y helados,Ana,Trujillo,Owner,Avda. de la Constitución,2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
ANTON,Antonio Moreno Taquería,Antonio,Moreno,Owner,Mataderos,2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
AROUT,Around the Horn,Thomas,Hardy,Sales Representative,Hanover Sq.,120,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
BERGS,Berglunds snabbköp,Christina,Berglund,Order Administrator,Berguvsvägen,8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...,...
WHITC,White Clover Markets,Karl,Jablonski,Owner,- th Ave. S. Suite B,305,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
VICTE,Victuailles en stock,Mary,Saveley,Sales Agent,", rue du Commerce",2,Lyon,None,69004,France,78.32.54.86,78.32.54.87
WILMK,Wilman Kala,Matti,Karttunen,Owner/Marketing Assistant,Keskuskatu,45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


In [12]:
#

4. Changing most of the columns data type to its suitable data type.

5. Postal code must be cleaned up

# 5. Correct the data types of each columns

# 6. Remove duplicates

# 7. Normalize data & Data formatting

# 8. Create new column for better analysis (if needed)

# 9. Analyzing data 

# 10. Export clean data

# 11. Close the connection